In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

In [ ]:
model_name = "gpt-4o"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

In [ ]:
role = "travel agent"
company = "contoso travel"
responsibility = "booking flights"

In [ ]:
response = client.complete(
    messages=[
        SystemMessage(content="""You are an expert at creating AI agent assitants. 
You will be provided a company name, role, responsibilites and other
information that you will use to provide a system prompt for.
To create the system prompt, be descriptive as possible and provide a structure that a system using an LLM can better understand the role and responsibilites of the AI assistant."""),
        UserMessage(content=f"You are {role} at {company} that is responsible for {responsibility}."),
    ],
    model=model_name,
    # Optional parameters
    temperature=1.,
    max_tokens=1000,
    top_p=1.
)

print(response.choices[0].message.content)

In [13]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

In [15]:

# Create the agents.
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)
assistant = AssistantAgent(
    name="assistant",
    model_client=client,
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=False,
)
user_proxy = UserProxyAgent("user_proxy", input_func=input)  # Use input() to get user input from console.

# Create the termination condition which will end the conversation when the user says "APPROVE".
termination = TextMentionTermination("APPROVE")

# Create the team.
team = RoundRobinGroupChat([assistant, user_proxy], termination_condition=termination)

# Run the conversation and stream to the console.
stream = team.run_stream(task="Write a 4-line poem about the ocean.")
# Use asyncio.run(...) when running in a script.
await Console(stream)

---------- user ----------
Write a 4-line poem about the ocean.
---------- assistant ----------
Beneath the waves, where secrets lie,  
A realm of blue touches the sky.  
Whispers of the tide in gentle sway,  
The ocean’s heart calls us to play.  
---------- user_proxy ----------
ocean 11
---------- assistant ----------
"Ocean's Eleven" is a popular heist film released in 2001, directed by Steven Soderbergh. It features an ensemble cast, including George Clooney, Brad Pitt, Matt Damon, and Julia Roberts. The plot revolves around Danny Ocean (played by Clooney) and his team of eleven skilled con artists who plan an elaborate heist to rob three Las Vegas casinos simultaneously.

The film is known for its stylish direction, clever dialogue, and the chemistry among the cast members. If you're looking to explore similar themes or want to know more about the film's sequels, just let me know!
---------- user_proxy ----------
no
---------- assistant ----------
I apologize for any misunderstand

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a 4-line poem about the ocean.', type='TextMessage'), TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=33, completion_tokens=41), content='Beneath the waves, where secrets lie,  \nA realm of blue touches the sky.  \nWhispers of the tide in gentle sway,  \nThe ocean’s heart calls us to play.  ', type='TextMessage'), UserInputRequestedEvent(source='user_proxy', models_usage=None, request_id='8e7a47d1-a6a7-41b1-a97e-67f49055b52b', content='', type='UserInputRequestedEvent'), TextMessage(source='user_proxy', models_usage=None, content='ocean 11', type='TextMessage'), TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=85, completion_tokens=125), content='"Ocean\'s Eleven" is a popular heist film released in 2001, directed by Steven Soderbergh. It features an ensemble cast, including George Clooney, Brad Pitt, Matt Damon, and Julia Roberts. The plot revolves around Da